In [3]:
import pandas as pd
import numpy as np

contract = pd.read_csv('Dataset/combined_data.csv', encoding='utf-8')

/var/folders/8r/nxvv77y10g556swh249ds84w0000gn/T/ipykernel_38672/2963632914.py:4: DtypeWarning: Columns (0,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  contract = pd.read_csv('Dataset/combined_data.csv', encoding='utf-8')


In [2]:
def get_future_data(df, date, code):
    # 读取数据，使用GBK编码

    # 将日期列转换为datetime类型，以确保可以进行日期比较
    df['time'] = pd.to_datetime(df['time'])
    # 替换'--'为NaN
    df.replace('--', np.nan, inplace=True)

    # 将输入的日期字符串也转换为datetime类型
    input_date = pd.to_datetime(date)

    # 筛选符合输入日期和代码的行
    filtered_data = df[(df['time'] == input_date) & (df['ths_future_code_future_processed'] == code)]

    # 检查筛选结果是否为空
    if filtered_data.empty:
        return "No data available for the given date and code."

    # 获取所需的列信息，如果'rate'是null，则使用'fee'
    result = filtered_data.apply(lambda row: {
        'code': row['ths_future_code_future_processed'],
        'contract_multiplier': int(row['ths_contract_multiplier']),
        'deposit': float(row['ths_trade_deposit_future']),
        'transactionfee': float(
            row['ths_transaction_procedure_rate_future'] if pd.isna(row['ths_transaction_procedure_rate_future']) or
                                                            row['ths_transaction_procedure_rate_future'] == 0.0 else
            row['ths_transaction_procedure_fee_future']),

        'opentime': row['ths_open_time_night_future'] if pd.notnull(row['ths_open_time_night_future']) else row[
            'ths_open_time_day_future'],

    }, axis=1)

    # 返回结果
    return result.iloc[0]  # 假设每个日期和代码的组合只有一行数据

def calculate_trade_return(row):
    # 计算基础收益，计算方法为手数 * （收盘价- 开盘价） * 点数
    # 注意需要读取未进行换月连续化的数据进行计算，以提高准确性
    
    row['Return'] = row['Direction'] * (row['Close_Price'] - row['Open_Price']) * (get_future_data(contract,row['Open_Time'],row['Asset'])['contract_multiplier'])
    
    # 判断手续费是固定还是按比例,默认手续费开双边 *2
    if get_future_data(contract,row['Open_Time'],row['Asset'])['transactionfee'] < 1:
        # 如果手续费按比例收，计算方法为（开价+收价）* 点数 * 手数绝对值 * 比例
        commission = ((get_future_data(contract,row['Open_Time'],row['Asset'])['transactionfee'] 
                      * get_future_data(contract,row['Open_Time'],row['Asset'])['contract_multiplier'] * (row['Open_Price'] + row['Close_Price']))
                      * abs(row['Direction'])*2)
    else:  # 固定手续费，直接用手续费值 * 手数绝对值 * 2即可
        commission = get_future_data(contract,row['Open_Time'],row['Asset'])['transactionfee'] * 2 * abs(row['Direction'])
    # 最终收益 = 基础收益 - 手续费
    row['Return'] = row['Return'] - commission
    return row

In [4]:
signals = pd.read_csv('records/2024-04-22_10-23-38/signals.csv')
df = pd.DataFrame(signals,
                    columns=['Asset', 'Open_Time', 'Open_Price', 'Direction', 'Close_Time', 'Close_Price'])

# 将平仓时间的格式转化为时间戳
df['Close_Time'] = pd.to_datetime(df['Close_Time'])

# 按照平仓时间排序， 以平仓时间作为一次交易结束并结算的时间
df = df.sort_values(by='Close_Time')

# 计算每笔交易的收益（转化单位为人民币，考虑手续费）
df = df.apply(calculate_trade_return, axis=1)